In [1]:
from __future__ import absolute_import, division
from __future__ import print_function, unicode_literals
import pints.toy as toy
import pints
import numpy as np
import logging
import math
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from numpy import inf
import copy 
import pickle
import time
import CMA as CMA

In [5]:
def CMA_on_HyperRect_Homogeneous(opt, 
                 problem,
                 sampler,
                 sampler_x0, 
                 n_chains=1,
                 chain_size=100,
                 need_sensitivities=False, 
                eval_fun=['ESS']):
    
    optimizer_best_fxs = []
    optimizer_best_xs = []
    
    optimizer_best_fx = np.inf
    optimizer_best_x = 0
    
    for _ in range(50):
        
        # Getting the samples of hyper-parameters for the samplers
        optimizer_xs = opt.ask()
        
        # Saving the score of each sample of hyper-parameters
        optimizer_fxs = []
        
        # Evaluate performance for each hyper-parameter configuration
        for x in optimizer_xs:
            
            our_x = x
            
            # Initialise function evaluations and matrix for chains
            function_evaluations = 0 
            chains = []
            
            # Return array of samples for each chain
            for i in range(n_chains):

                # Create sampler object and set hyperparameter
                curr_x0 = sampler_x0[i]
                mcmc = sampler(curr_x0)
                mcmc.set_hyper_parameters([our_x, True])
                i_chain = []
                
                # Update until we have ``chain_size`` samples
                # Update function evaluations each time we use ask(),tell()
                while len(i_chain) < chain_size:
                    x = mcmc.ask()
                    if need_sensitivities:
                        fx, grad = problem.evaluateS1(x)
                        sample = mcmc.tell((fx, grad))
                    else:
                        fx = problem.evaluateS1(x)
                        sample = mcmc.tell(fx)
                    function_evaluations += 1
                    if sample is not None:
                        i_chain.append(sample)
                
                # Append ith chain to list of chains       
                chains.append(i_chain)
                
            chains = np.array(chains, copy=True)
            optimizer_fx = 0
            # Calculate the score of the sampler with the given hyper-parameters
            # Get the KL if requested
            if 'KL' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                optimizer_fx = kl/len(chains)                
            
            elif 'KL-ITER' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                avg_kl = kl/len(chains)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = avg_kl * avg_iteration_count
            
            # Get the ESS if requested
            elif 'ESS' in eval_fun:
                ess = np.zeros(chains[0].shape[1])
                for chain in chains:
                    ess += np.array(pints._diagnostics.effective_sample_size(chain))
                ess /= len(chains)
                ess = np.min(ess)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = math.sqrt(avg_iteration_count) / ess
                
            optimizer_fxs.append(optimizer_fx)
        opt.tell(optimizer_fxs)
        
        optimizer_best_fxs.append(opt.fbest())
        optimizer_best_xs.append(opt.xbest())
        
        if opt.fbest() < optimizer_best_fx:
            optimizer_best_fx = opt.fbest()
            optimizer_best_x = opt.xbest()
    
    print(optimizer_best_fx)
    print(optimizer_best_x)
    print()
    
    return optimizer_best_fxs, optimizer_best_xs

In [4]:
rosenbrock = pints.toy.RosenbrockLogPDF()
eggshape = pints.toy.SimpleEggBoxLogPDF(2,4)
Logistic = pints.toy.LogisticModel()

problem = rosenbrock

# Define the starting points of the samplers
sampler_rosenbrock_x0 = xs = [
    [0,0],
    [3,8],
    [2,4],
]

sampler_egg_x0 = xs = [
    [-10,-10],
    [0,0],
    [10,10],
]

sampler_x0 = sampler_rosenbrock_x0

# Define the starting point of the optimzer
dummy = pints.SliceHyperrectanglesMCMC([2, 4])
optimizer_x0 = [dummy.width()[0], dummy.width()[1]]

boundary = pints.RectangularBoundaries([0,0],[20,20])

sampler = pints.SliceHyperrectanglesMCMC

for i in range(5):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_HyperRect_Homogeneous(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=100, eval_fun=['ESS'])

Best Score
1.7127249311275041
Best Hyper-Parameters
[9.49495527 8.44916381]

Best Score
1.834308470251693
Best Hyper-Parameters
[0.26063598 0.05373724]

Best Score
1.9258250416771152
Best Hyper-Parameters
[0.10932187 0.48717175]

Best Score
1.7694521587367684
Best Hyper-Parameters
[8.71016373 8.24312221]

Best Score
1.8256559039416769
Best Hyper-Parameters
[7.90387507 9.64259045]



In [3]:
# Define the starting point of the optimzer
dummy = pints.SliceHyperrectanglesMCMC([2, 4])
optimizer_x0 = [dummy.width()[0], dummy.width()[1]]
print(optimizer_x0)

AttributeError: module 'pints' has no attribute 'SliceHyperrectanglesMCMC'

## Results for the EggShape problem

In [ ]:
0.6633780365310231
[8.53837666 9.14565402]

0.671087654591173
[10.17517565  9.21948061]

0.6847802080854595
[9.43201469 8.6194252 ]

0.684552052087733
[8.35683414 9.9708093 ]

0.6862461492720475
[10.09632839  8.42542171]

## Results for the Rosenbrock Problem

In [6]:
1.7127249311275041
[9.49495527 8.44916381]

1.834308470251693
[0.26063598 0.05373724]

1.9258250416771152
[0.10932187 0.48717175]

1.7694521587367684
[8.71016373 8.24312221]

1.8256559039416769
[7.90387507 9.64259045]

SyntaxError: invalid syntax (<ipython-input-6-f244d5ce9110>, line 1)

## Logistic Model and results

In [7]:
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

class ExampleModel(pints.ForwardModel):
    
    def __init__(self):
        self.model = toy.LogisticModel()
    def simulate(self, x, times):
        return self.model.simulate([x[0]/1000, x[1]], times)
    def simulateS1(self, x, times):
        values, gradient = self.model.simulateS1([x[0]/1000, x[1]], times)
        return values, gradient
    def n_parameters(self):
        return 2

# Then create an instance of our new model class
model = ExampleModel()

In [8]:
# Create some toy data
real_parameters = [0.015*10000, 500]
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)
real_parameters = np.array(real_parameters + [noise])

# Get properties of the noise sample
noise_sample_mean = np.mean(values - org_values)
noise_sample_std = np.std(values - org_values)

# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
log_likelihood = pints.GaussianLogLikelihood(problem)

# Create a uniform prior over both the parameters and the new noise variable
log_prior = pints.UniformLogPrior(
    [0.01*10000, 400, noise * 0.1],
    [0.02*10000, 600, noise * 100],
)

# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Choose starting points for 3 mcmc chains
num_chains = 3
sampler_logistic_x0 = [
    real_parameters * 1.01,
    real_parameters * 0.99,
    real_parameters * 1.01
]

In [9]:
problem = log_posterior

sampler_x0 = sampler_logistic_x0

# Define the starting point of the optimzer
dummy = pints.SliceHyperrectanglesMCMC([2, 4, 10])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.width()[2]]

boundary = pints.RectangularBoundaries([0,0,0],[20,20,20])

sampler = pints.SliceHyperrectanglesMCMC

for i in range(5):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_HyperRect_Homogeneous(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=100, eval_fun=['ESS'])

/home/naunauyoh/anaconda3/lib/python3.7/site-packages/pints/_log_likelihoods.py:476: RuntimeWarning: invalid value encountered in log
  - np.sum(error**2, axis=0) / (2 * sigma**2))


1.5602994194803705
[5.61401202 4.22481883 0.75674336]

1.685472660356954
[7.34162696 4.48995497 0.74637457]

1.654779618231904
[8.6941372  3.66687751 1.00450451]

1.4898493920617981
[13.46896167  4.57889999  1.00920944]

1.7148956776000917
[11.92137682  7.60302896  0.91595197]



In [ ]:
1.5602994194803705
[5.61401202 4.22481883 0.75674336]

1.685472660356954
[7.34162696 4.48995497 0.74637457]

1.654779618231904
[8.6941372  3.66687751 1.00450451]

1.4898493920617981
[13.46896167  4.57889999  1.00920944]

1.7148956776000917
[11.92137682  7.60302896  0.91595197]